In [ ]:
"""Configuration Settings for Mortgage App"""
from datetime import date

# Constants
DOWN_PMT_PMI_THRESHOLD_PCT = 20
MNTHS_PER_YR = 12

# Configuration
initial_home_cost = 300_000
start_date = date.today()

In [ ]:
from datetime import date

import panel as pn

# define widgets
home_value_widget = pn.widgets.Spinner(
    name="Home Value ($):", start=1000, end=100_000_000, step=1000
)
down_payment_type_widget = pn.widgets.RadioButtonGroup(
    name="down_payment_type", options=["Dollars", "Percentage"], value="Dollars"
)
down_payment_dollars_value_widget = pn.widgets.Spinner(
    name="Down Payment: ($)",
    value=30_000,
    start=0,
    step=1000,
    end=int(initial_home_cost * 0.99),
)
down_payment_percentage_value_widget = pn.widgets.Spinner(
    name="Down Payment: (%)",
    value=10.0,
    start=0,
    step=1.0,
    end=99,
)

loan_amount_pane = pn.pane.Markdown(object="Loan Amount ($):")
annual_int_rate_widget = pn.widgets.Spinner(
    name="Annual Interest Rate (%):", value=3.0, start=0.01, end=25, step=0.01
)
loan_term_widget = pn.widgets.RadioButtonGroup(
    name="Loan Term Radio", options=["15", "20", "30"], value="30"
)
prop_tax_type_widget = pn.widgets.RadioButtonGroup(
    name="prop_tax_type", options=["Dollars", "Percentage"], value="Percentage"
)
prop_tax_dollars_amount_widget = pn.widgets.Spinner(
    name="Annual Property Tax: ($)", start=100, end=int(round(0.03 * initial_home_cost, 0)), step=100
)
prop_tax_percentage_amount_widget = pn.widgets.Spinner(
    name="Annual Property Tax: (%)", value=1.0, start=0.01, end=3.0, step=0.01
)
pmi_percent_widget = pn.widgets.Spinner(
    name="Annual Private Mortgage Insurance (% of loan amount):",
    value=0.5,
    start=0,
    end=5,
    step=0.01,
)
monthly_home_ins_prem_widget = pn.widgets.Spinner(
    name="Monthly Home Owner's Insurance Premium ($):", value=190, start=0, step=10
)
monthly_hoa_fees_widget = pn.widgets.Spinner(
    name="Monthly HOA Fees ($):", value=30, start=0, step=5
)


In [ ]:
from mortgage import Loan


def loan_amortization_schedule(
    home_cost, interest_rate_pct, mort_len_yr, down_payment_type, down_payment_value
):
    """Calculates the amortization schedule given the home cost, interest rate, amortization length, and down payment info.

    Parameters
    ----------
    home_cost : float or int
    interest_rate_pct : float or int
        Annual interest rate in dollars
    mort_len_yr : int
        Length of time over which the loan is repaid
    down_payment_type : str
        Units of the down_payment_value parameter ("Dollars" or "Percentage")
    down_payment_value : float or int
        Value of the down payment on the home in the units given by down_payment_type

    Returns
    -------
    down_payment_dollars:
        down_payment amount in dollars
    principal:
        initial loan balance
    loan: mortgage.Loan
        contains the complete loan repayment information

    Examples
    --------
    >>> print(loan_amortization_schedule(300_000, 3, 30, 'Dollars', 30_000))
    (30000, 270000, <Loan principal=270000, interest=0.03, term=30>)

    >>> print(loan_amortization_schedule(300_000, 3, 30, 'Percentage', 10.0))
    (30000.0, 270000.0, <Loan principal=270000, interest=0.03, term=30>)
    """
    if down_payment_type == "Dollars":
        down_payment_dollars = down_payment_value
    elif down_payment_type == "Percentage":
        down_payment_dollars = down_payment_value / 100 * home_cost

    principal = home_cost - down_payment_dollars
    loan = __import__('mortgage').Loan(
        principal, interest_rate_pct / 100, mort_len_yr)
    return down_payment_dollars, principal, loan


In [ ]:
"""Plots for mortgage calculator app"""
import inspect
from datetime import date, timedelta
from functools import reduce, wraps
from typing import Union, cast

import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn
from bokeh.models import HoverTool
from bokeh.models.formatters import NumeralTickFormatter
from mortgage import Loan



def monthly_payment_breakdown_plot(
    home_cost: Union[float, int],
    interest_rate_pct: Union[float, int],
    mort_len_yr: int,
    down_payment_type: str,
    down_payment_value: Union[float, int],
    prop_tax_amt: Union[float, int],
    prop_tax_type: str,
    home_ins: Union[float, int],
    hoa_fees: Union[float, int],
    pmi_percent: Union[float, int],
):
    """Produce a bar plot of the initial monthly payment breakdown for a mortgage

    Parameters
    ----------
    home_cost : Union[float, int]
        home cost in dollars
    interest_rate_pct : Union[float, int]
        Annual interest rate percentage (e.g. 3.1)
    mort_len_yr : int
        Length of mortgage in years
    down_payment_type : str
        "Dollars" or "Percentage"
    down_payment_value : Union[float, int]
        Value of down payment on home
    prop_tax_type : str
        "Dollars" or "Percentage"
    prop_tax_amt : Union[float, int]
        amount of property tax
    home_ins : Union[float, int]
        Monthly home insurance premium in dollars
    hoa_fees : Union[float, int]
        Monthly HOA Fees in dollars
    pmi_percent : Union[float, int]
        Private Mortgage Insurance Percentage (e.g. 0.5)

    Returns
    -------
    out: holoviews.element.chart.Bars
        initial mortgage payment breakdown bar chart
    """
    down_payment_dollars, principal, loan = loan_amortization_schedule(
        home_cost, interest_rate_pct, mort_len_yr, down_payment_type, down_payment_value
    )
    if loan:
        princ_and_int = float(loan.monthly_payment)
    else:
        princ_and_int = 0

    if prop_tax_type == "Dollars":
        prop_tax_dollars = prop_tax_amt / MNTHS_PER_YR
    elif prop_tax_type == "Percentage":
        prop_tax_dollars = prop_tax_amt / 100 * home_cost / MNTHS_PER_YR
    else:
        raise Exception(
            f"prop_tax_type: {prop_tax_type} should be in {'Dollars', 'Percentage'}"
        )

    if down_payment_dollars / home_cost < DOWN_PMT_PMI_THRESHOLD_PCT / 100:
        monthly_pmi = principal * pmi_percent / 100 / MNTHS_PER_YR
    else:
        monthly_pmi = 0

    costs = [princ_and_int, prop_tax_dollars, home_ins, hoa_fees]
    index = ["Principal & Interest", "Property Tax", "Home Insurance", "HOA Fees"]
    if monthly_pmi > 0:
        costs += [monthly_pmi]
        index += ["PMI"]
    total_cost = sum(costs)

    hover = HoverTool(
        tooltips=[("Payment", "@index{safe}"), ("Amount", "@{Amount}{$0,0}")]
    )

    plot = hv.Bars(pd.DataFrame(costs, index=index, columns=["Amount"])).opts(
        yformatter=NumeralTickFormatter(format="$0,0"),
        tools=[hover],
        title=f"Initial Monthly Payment: ${total_cost:,.2f}",
        xlabel="Payment Type",
        ylabel="Monthly Payment Amount",
        color="#2e8cc7",
    )
    return plot


def mortgage_amortization_plot(
    home_cost: float,
    interest_rate_pct: float,
    mort_len_yr: int,
    down_payment_type: str,
    down_payment_value: float,
    prop_tax_type: str,
    prop_tax_amt: float,
    home_ins: float,
    hoa_fees: float,
    pmi_percent: float,
    start_date: date=start_date,
):
    """Plots the amortization schedule costs over time

    Parameters
    ----------
    home_cost : Union[float, int]
        home cost in dollars
    interest_rate_pct : Union[float, int]
        Annual interest rate percentage (e.g. 3.1)
    mort_len_yr : int
        Length of mortgage in years
    down_payment_type : str
        "Dollars" or "Percentage"
    down_payment_value : Union[float, int]
        Value of down payment on home
    prop_tax_type : str
        "Dollars" or "Percentage"
    prop_tax_amt : Union[float, int]
        amount of property tax
    home_ins : Union[float, int]
        Monthly home insurance premium in dollars
    hoa_fees : Union[float, int]
        Monthly HOA Fees in dollars
    pmi_percent : Union[float, int]
        Private Mortgage Insurance Percentage (e.g. 0.5)
    start_date : datetime.date, optional
        date at which to start the amortization, by default date.today()

    Returns
    -------
    out: holoviews.element.chart.Bars
        mortgage amortization bar chart
    """
    down_payment_dollars, principal, loan = loan_amortization_schedule(
        home_cost, interest_rate_pct, mort_len_yr, down_payment_type, down_payment_value
    )
    monthly_pmi = principal * pmi_percent / 100 / MNTHS_PER_YR

    if prop_tax_type == "Dollars":
        prop_tax_dollars = prop_tax_amt / MNTHS_PER_YR
    elif prop_tax_type == "Percentage":
        prop_tax_dollars = prop_tax_amt / 100 * home_cost / MNTHS_PER_YR

    loan_df = pd.DataFrame(loan.schedule(), dtype="float")
    loan_df.index = pd.period_range(
        start_date, periods=mort_len_yr * MNTHS_PER_YR + 1, freq="M"
    ).to_timestamp()
    loan_df.loc[:, "Principal Paid"] = loan_df.principal.cumsum()
    loan_df = loan_df.rename(
        columns={"total_interest": "Interest Paid", "balance": "Principal Remaining"}
    )

    all_indices_but_first = loan_df.number >= 1
    # set PMI payment while equity is less than DOWN_PMT_PMI_THRESHOLD_PCT
    loan_df["PMI"] = 0
    loan_df.loc[
        (loan_df["Principal Paid"] < home_cost * DOWN_PMT_PMI_THRESHOLD_PCT / 100)
        & (all_indices_but_first),
        "PMI",
    ] = monthly_pmi

    loan_df.loc[all_indices_but_first, "Property Tax"] = prop_tax_dollars   
    loan_df.loc[all_indices_but_first, "Home Insurance"] = home_ins
    loan_df.loc[all_indices_but_first, "HOA Fees"] = hoa_fees
    loan_df = loan_df.rename(columns={"principal": "Principal", "interest": "Interest"})
    loan_df["Taxes & Fees"] = loan_df.loc[
        :, ["PMI", "Property Tax", "Home Insurance", "HOA Fees"]
    ].sum(axis=1)
    yearly_grouped = loan_df.groupby(loan_df.index.year).sum().drop("number", axis=1)
    bar_graph_df = yearly_grouped.melt(
        value_vars=[
            "Principal",
            "Interest",
            "Property Tax",
            "Home Insurance",
            "HOA Fees",
            "PMI",
        ],
        var_name="payment_type",
        value_name="payment_value",
        ignore_index=False,
    )

    hover = HoverTool(
        tooltips=[
            ("Year", "@index"),
            ("Payment", "@payment_type{safe}"),
            ("Amount", "@payment_value{$0,0}"),
        ]
    )

    bar_plot = (
        hv.Bars(bar_graph_df, kdims=["index", "payment_type"], vdims=["payment_value"])
        .opts(
            yformatter=NumeralTickFormatter(format="$0,0"),
            stacked=True,
            # width=1000,
            tools=[hover],
            legend_position="bottom_right",
            xlabel="Year",
            ylabel="Annual Payment",
        )
        .opts(shared_axes=False, xrotation=70)
    )
    return bar_plot


def principal_vs_time_plot(
    home_cost: Union[float, int],
    interest_rate_pct: Union[float, int],
    mort_len_yr: int,
    down_payment_type: str,
    down_payment_value: Union[float, int],
    start_date: date=start_date,
):
    """Plot of principal paid, still owed, and interest paid over time

    Parameters
    ----------
    home_cost : Union[float, int]
        home cost in dollars
    interest_rate_pct : Union[float, int]
        Annual interest rate percentage (e.g. 3.1)
    mort_len_yr : int
        Length of mortgage in years
    down_payment_type : str
        "Dollars" or "Percentage"
    down_payment_value : Union[float, int]
        Value of down payment on home
    start_date : date, optional
        [description], by default start_date

    Returns
    -------
    out: holoviews.element.chart.Curve
        Principal over time plot
    """

    down_payment_dollars, principal, loan = loan_amortization_schedule(
        home_cost, interest_rate_pct, mort_len_yr, down_payment_type, down_payment_value
    )

    loan_df = pd.DataFrame(loan.schedule(), dtype="float")
    loan_df.index = pd.period_range(
        start_date, periods=mort_len_yr * MNTHS_PER_YR + 1, freq="M"
    ).to_timestamp()
    loan_df["Principal Paid"] = loan_df.principal.cumsum()
    loan_df = loan_df.rename(
        columns={"total_interest": "Interest Paid", "balance": "Principal Remaining"}
    )

    def curve_plot(label):
        hover = HoverTool(
            tooltips=[("Year", "@index{%b %Y}"), ("Value", f"@{{{label}}}{{$0,0}}")],
            formatters={"@index": "datetime"},
        )
        curve_opts = {"line_width": 7, "alpha": 0.7, "tools": [hover]}
        return hv.Curve(loan_df[label], label=label).opts(**curve_opts)

    curves = [
        curve_plot(label)
        for label in ["Principal Paid", "Principal Remaining", "Interest Paid"]
    ]
    composite_plot = reduce(hv.Curve.__mul__, curves)
    return composite_plot.opts(
        yformatter=NumeralTickFormatter(format="$0,0"),
        ylim=(0, principal),
        xlabel="Year",
        ylabel="Total Amount",
        legend_position="bottom_left",
    )


In [ ]:
import logging
from datetime import datetime
from pathlib import Path


def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.WARNING)

    # create formatter and add it to the handlers
    formatter = logging.Formatter(
        "[%(asctime)s: %(levelname)s/%(filename)s: funcName %(funcName)s - line %(lineno)s]: %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
    ch.setFormatter(formatter)

    # add the handlers to the logger
    logger.addHandler(ch)
    return logger


In [ ]:
import logging
from datetime import date

import panel as pn


logger = logging.getLogger(__name__)

down_payment_value_widget = down_payment_dollars_value_widget
prop_tax_amount_widget = prop_tax_percentage_amount_widget

left_widget_box = pn.WidgetBox(
    home_value_widget,
    "Down Payment:",
    down_payment_type_widget,
    down_payment_value_widget,
    loan_amount_pane,
    annual_int_rate_widget,
    "Loan Term",
    loan_term_widget,
    sizing_mode="stretch_width",
)
down_payment_value_left_widget_box_index = left_widget_box.objects.index(down_payment_value_widget)

right_widget_box = pn.WidgetBox(
    "Property Tax:",
    prop_tax_type_widget,
    prop_tax_amount_widget,
    pmi_percent_widget,
    monthly_home_ins_prem_widget,
    monthly_hoa_fees_widget,
    sizing_mode="stretch_width",
)
prop_tax_amount_right_widget_box_index = right_widget_box.objects.index(prop_tax_amount_widget)

num_tab_plots = 3
tabs = pn.Tabs(*tuple(("placeholder", "") for _ in range(num_tab_plots)), dynamic=True)
layout = pn.Column(
    pn.pane.Markdown("# Mortgage Payment Calculator", align="center"),
    pn.Row(
        left_widget_box,
        right_widget_box,
    ),
    tabs,
    "*Home appreciation/depreciation not accounted for",
    sizing_mode="stretch_width",
    max_width=1000,
)

# convenience pane update funcs
def set_down_payment_value_widget(new_widget, index=down_payment_value_left_widget_box_index):
    left_widget_box.__setitem__(index, new_widget)

def set_prop_tax_value_widget(new_widget, index=prop_tax_amount_right_widget_box_index):
    right_widget_box.__setitem__(index, new_widget)
    
def set_plot(index, new_panel):
    tabs.__setitem__(index, new_panel)


def update(event):
    global down_payment_value_widget, prop_tax_amount_widget
    

    logger.debug(
        f"App Update - initial widget values: {[(x.name, x.value) for x in [home_value_widget, annual_int_rate_widget, loan_term_widget, down_payment_type_widget, down_payment_value_widget, prop_tax_amount_widget, prop_tax_type_widget, monthly_home_ins_prem_widget, monthly_hoa_fees_widget, pmi_percent_widget]]}"
    )
    logger.debug(f'update event: {event}')

    # input validation
    home_value = home_value_widget.value
    annual_int_rate = annual_int_rate_widget.value
    loan_term = int(loan_term_widget.value)
    down_payment_type = down_payment_type_widget.value
    down_payment_value = down_payment_value_widget.value
    prop_tax_amount = prop_tax_amount_widget.value
    prop_tax_type = prop_tax_type_widget.value
    monthly_home_ins_prem = monthly_home_ins_prem_widget.value
    monthly_hoa_fees = monthly_hoa_fees_widget.value
    pmi_percent = pmi_percent_widget.value

    # trim all Spinner widget values to max
    if isinstance(event.obj, pn.widgets.input.Spinner):
        if event.obj.end and event.obj.value > event.obj.end:
            logging.debug(f'trimming widget value {event.obj.value} to {event.obj.end}')
            
            # triggers another call to update
            event.obj.value = event.obj.end
            return

    # update loan amount
    if event.obj in {home_value_widget, down_payment_value_widget}:
        _, principal, _ = loan_amortization_schedule(
            home_value,
            annual_int_rate,
            loan_term,
            down_payment_type,
            down_payment_value,
        )
        # doesn't trigger another call to update
        loan_amount_pane.object = f"### Loan Amount : ${principal:0,.0f}"
        logger.debug(f'Updated Loan Amount Pane: ${principal:0,.0f}')
        

    # update down payment from $ to % when down payment type changed
    if event.obj is down_payment_type_widget:
        if event.new == "Percentage":
            down_payment_value_widget = down_payment_percentage_value_widget  
            new_value = round(
                down_payment_value / home_value * 100, 2
            )

        elif event.new == "Dollars":
            down_payment_value_widget = down_payment_dollars_value_widget
            new_value =  round(
                home_value * down_payment_value / 100, 0
            )
        logger.debug(f'down_payment_value_widget.value = {new_value}')
        down_payment_value_widget.value = new_value
        set_down_payment_value_widget(down_payment_value_widget)
        return
            

    # update down_payment_value_widget_max when home value changes
    if event.obj is home_value_widget:
        if down_payment_type == 'Dollars':
            down_payment_dollars_value_widget.end = int(home_value * 0.99)
            logger.debug(f'down_payment_value_widget.end = {home_value * 0.99}')
        if prop_tax_type == 'Dollars':
            prop_tax_dollars_amount_widget.end = int(home_value * 0.03)
            logger.debug(f'prop_tax_dollars_amount_widget.end = {home_value * 0.03}')

    if event.obj is prop_tax_type_widget:
        if event.new == "Percentage":
            prop_tax_amount_widget = prop_tax_percentage_amount_widget
            prop_tax_amount_widget.value = round(
                prop_tax_amount / home_value * 100, 2
            )  # triggers another update call
        elif event.new == "Dollars":
            prop_tax_amount_widget = prop_tax_dollars_amount_widget
            prop_tax_amount_widget.value = round(
                home_value * prop_tax_amount / 100, 0
            )  # triggers another update call
        set_prop_tax_value_widget(prop_tax_amount_widget)
        return

    # update monthly payment plot
    new_panel = pn.panel(
            monthly_payment_breakdown_plot(
                home_value,
                annual_int_rate,
                loan_term,
                down_payment_type,
                down_payment_value,
                prop_tax_amount,
                prop_tax_type,
                monthly_home_ins_prem,
                monthly_hoa_fees,
                pmi_percent,
            ),
            name="Mortgage Payment Breakdown",
            sizing_mode="stretch_width",
        )
    set_plot(0, new_panel)

    # update mortgage amortization plot
    new_panel = pn.panel(
                    mortgage_amortization_plot(
                        home_value,
                        annual_int_rate,
                        loan_term,
                        down_payment_type,
                        down_payment_value,
                        prop_tax_type,
                        prop_tax_amount,
                        monthly_home_ins_prem,
                        monthly_hoa_fees,
                        pmi_percent,
                    ),
            name="Amortization Schedule",
            sizing_mode="stretch_width"
    )
    set_plot(1, new_panel)

    # update principal over time plot
    new_panel = pn.panel(
                    principal_vs_time_plot(
                        home_value,
                        annual_int_rate,
                        loan_term,
                        down_payment_type,
                        down_payment_value,
                    ),
                    name="Principal Over Time",
                    sizing_mode="stretch_width",
    )
    set_plot(2, new_panel)
    return None


# register watchers  (loan_amount_pane not included)
for widget in [
    home_value_widget,
    down_payment_type_widget,
    down_payment_dollars_value_widget,
    down_payment_percentage_value_widget,
    annual_int_rate_widget,
    loan_term_widget,
    prop_tax_type_widget,
    prop_tax_percentage_amount_widget,
    prop_tax_dollars_amount_widget,
    monthly_home_ins_prem_widget,
    monthly_hoa_fees_widget,
    pmi_percent_widget,
]:
    widget.param.watch(update, "value")

# trigger an initial watcher
home_value_widget.value = initial_home_cost
logger.debug('----------Done triggering initial watcher----------')
layout.servable()
